# Extraer datos con API last.fm POP

In [1]:
!pip install spotipy pandas requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [spotipy]


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

In [5]:
# --- 1. GUARDAR CREDENCIALES EN VARIABLES  ---
URL = 'https://dataspotifypop.com'
CLIENT_ID = 'b052486aab8747bab21a22091b794972'
CLIENT_SECRET = '1821327a005e41efbca43269b84a8474'

In [7]:
# --- 2. AUTENTICARSE  ---
try:
    auth_manager = SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )

    sp = spotipy.Spotify(auth_manager=auth_manager)
    print("Autenticación con Spotify exitosa.")
    
except Exception as e:
    print(f"Error de autenticación: {e}")
    exit()

Autenticación con Spotify exitosa.


In [18]:
# Sacar datos de genero pop del 2010
datos = sp.search(q="genre:pop year:2010", type='track', limit=50, offset=0)


In [12]:
# mirar solo el primer track a ver como estan prganizados los datos y que podemos obtener
primer_track = datos['tracks']['items'][0]
primer_track

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6jJ0s89eD6GaHleKKya26X'},
    'href': 'https://api.spotify.com/v1/artists/6jJ0s89eD6GaHleKKya26X',
    'id': '6jJ0s89eD6GaHleKKya26X',
    'name': 'Katy Perry',
    'type': 'artist',
    'uri': 'spotify:artist:6jJ0s89eD6GaHleKKya26X'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'KW',
   'EG',
   'MA',
   'DZ',
   'TN',
 

In [ ]:
# claves principales
print(datos.keys())

# claves dentro de tracks
print(datos['tracks'].keys())

# claves dentro del primer track
print(datos['tracks']['items'][0].keys())

dict_keys(['tracks'])
dict_keys(['href', 'limit', 'next', 'offset', 'previous', 'total', 'items'])


dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [22]:
GENRE_TARGET = 'pop' 
YEARS_LIST = [2010, 2015, 2020, 2025] 
SPOTIFY_MAX_LIMIT = 1000

results = []
artistas = []

In [23]:


print("--- Inicio de la Extracción Masiva de Spotify ---")

# --- BUCLE PRINCIPAL DE EXTRACCIÓN: Iterar por año ---

for year in YEARS_LIST:
    print(f"\n>>>> Extrayendo datos para el género '{GENRE_TARGET}' en el año: {year}")
    
    offset = 0 # Reinicia el desplazamiento para cada año
    
    while offset < SPOTIFY_MAX_LIMIT:
        
        # 1. Definición de la consulta (género y año específicos)
        query = f'genre:{GENRE_TARGET} year:{year}'
        
        # 2. Llamada a la API de Spotify (SIN PAUSA DE TIEMPO)
        try:
            response = sp.search(q=query, limit=50, offset=offset, type='track,album')
        except Exception as e:
            print(f"Error en la API de Spotify en offset {offset}: {e}")
            break
        
        # --- PROCESAMIENTO DE TRACKS (Canciones) ---
        
        current_tracks = response['tracks']['items']
        if not current_tracks:
            break

        for item in current_tracks:
            artist_name = item['artists'][0]['name'] if item['artists'] else 'Artista Desconocido'
            artistas.append(artist_name)
            
            results.append({
                'uri_track': item['uri'],
                'name_track': item['name'],
                'name_artist': artist_name,
                'genre': GENRE_TARGET, 
                'type': 'track', 
                'year': str(year),
            })

        # --- PROCESAMIENTO DE ALBUMS (Solo en la primera página) ---
        
        if offset == 0 and 'albums' in response:
            for item in response['albums']['items']:
                artist_name = item['artists'][0]['name'] if item['artists'] else 'Artista Desconocido'
                artistas.append(artist_name)
                
                results.append({
                    'uri_track': item['uri'],
                    'name_track': item['name'],
                    'name_artist': artist_name,
                    'genre': GENRE_TARGET, 
                    'type': 'album', 
                    'year': str(year),
                })

        # --- AVANCE DE PÁGINA ---
        offset += 50 
        
        if offset % 100 == 0:
            print(f"   -> Offset actual: {offset}. Registros totales recolectados: {len(results)}")


# --- FIN DEL PROCESO ---
artistas_unicos = list(set(artistas)) 
print("\n--- Resumen de Extracción ---")
print(f"CONTEO FINAL: Total de registros (tracks + albums) recolectados: {len(results)}")
print(f"Artistas Únicos recolectados para la fase Last.fm: {len(artistas_unicos)}")

--- Inicio de la Extracción Masiva de Spotify ---

>>>> Extrayendo datos para el género 'pop' en el año: 2010
   -> Offset actual: 100. Registros totales recolectados: 101
   -> Offset actual: 200. Registros totales recolectados: 201
   -> Offset actual: 300. Registros totales recolectados: 301
   -> Offset actual: 400. Registros totales recolectados: 401
   -> Offset actual: 500. Registros totales recolectados: 501
   -> Offset actual: 600. Registros totales recolectados: 601
   -> Offset actual: 700. Registros totales recolectados: 701
   -> Offset actual: 800. Registros totales recolectados: 801
   -> Offset actual: 900. Registros totales recolectados: 901
   -> Offset actual: 1000. Registros totales recolectados: 1001

>>>> Extrayendo datos para el género 'pop' en el año: 2015
   -> Offset actual: 100. Registros totales recolectados: 1101
   -> Offset actual: 200. Registros totales recolectados: 1201
   -> Offset actual: 300. Registros totales recolectados: 1301
   -> Offset actual

In [ ]:
# --- CONFIGURACIÓN DE ARCHIVOS ---
SPOTIFY_OUTPUT_FILE = 'spotify_base_datos_final_pop.csv'
ARTISTS_OUTPUT_FILE = 'lastfm_unique_artists_pop.csv'

# --- 1. PROCESAR DATOS PRINCIPALES (Tracks y Albums) ---
print("\n--- Sección 3: Guardado de Datos ---")
# La variable 'results' viene de la Sección 2 ejecutada anteriormente.
print(f"Total de registros a guardar: {len(results)}") 

df_final = pd.DataFrame(results)

# Seleccionar y renombrar las columnas para cumplir con el enunciado
df_final_renombrado = df_final.rename(columns={
    'uri_track': 'Uri'
    'name_artist':'Artista',
    'genre': 'Genero_Musical_Seed',
    'type': 'Cancion_o_Album_Tipo',
    'name_track': 'Nombre_Cancion_o_Album',
    'year': 'Año_Lanzamiento'
})

# Eliminamos duplicados
initial_count = len(df_final_renombrado)
df_final_renombrado.drop_duplicates(subset=['Artista', 'Nombre_Cancion_o_Album', 'Año_Lanzamiento'], inplace=True)
print(f"-> Se eliminaron {initial_count - len(df_final_renombrado)} duplicados. Registros únicos: {len(df_final_renombrado)}")

# Guardar la base de datos principal de Spotify
df_final_renombrado.to_csv(SPOTIFY_OUTPUT_FILE, index=False)
print(f"Base de Datos de Spotify guardada como: '{SPOTIFY_OUTPUT_FILE}'")

# --- 2. PROCESAR ARTISTAS ÚNICOS PARA LAST.FM ---

# Crear el DataFrame de artistas únicos
# La variable 'artistas' viene de la Sección 2.
df_artists = pd.DataFrame(list(set(artistas)), columns=['artist_name'])
print(f"Total de artistas únicos para Last.fm: {len(df_artists)}")

# Guardar la lista de artistas únicos
df_artists.to_csv(ARTISTS_OUTPUT_FILE, index=False)
print(f"Lista de artistas para Last.fm guardada como: '{ARTISTS_OUTPUT_FILE}'")

# --- Muestra de la Base de Datos Final (CORRECCIÓN APLICADA AQUÍ) ---
columnas_muestra = ['Uri','Artista', 'Genero_Musical_Seed', 'Cancion_o_Album_Tipo', 'Nombre_Cancion_o_Album', 'Año_Lanzamiento']
print("\n--- Muestra de la Base de Datos Final (Primeras 5 Filas) ---")
# Usamos .head() en lugar de .to_markdown() para evitar el error de 'tabulate'
print(df_final_renombrado[columnas_muestra].head(5)) 

print("\n--- ¡Spotify y Guardado en CSVs Completados! ---")

SyntaxError: invalid syntax (87157114.py, line 15)